# MoSDeF Biomolecule Workflow


### Exercise Stages:
0. Setup enviroment on Google Colab
1. Import libraries
2. Load molecules
    * input parameters are protein, solvent, and box size. </br>
3. Pack a box of solvated protein
4. Load forcefield
    * use direct forcefields for each molecule type. </br>
5. Apply forcefield
    * apply to your molecule
6. Identify missing angles
    * modify xml file
7. Reload forcefield
8. Run simulation
    * run NVT hoomd simulation. </br>
9. Analysis
    * plot energy
---

## 0. Set up environment on Google Colab
---

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

In [ ]:
import condacolab
condacolab.check()

!conda install mamba
!git clone https://github.com/mosdef-hub/CECAM-MoSDeF-Workshop
!mamba env update -n base CECAM-MoSDeF-Workshop/environment.yml

!git clone https://github.com/mosdef-hub/gmso.git
%cd gmso
!git checkout -b 748-remove-gmso-xml-conversions
!pip install -e .
%cd ..

## 1. Import Libraries
---

In [ ]:
# Import Libraries
import mbuild as mb
import gmso

from pBuilder import Protein

## 2. Load Molecules
---

We can visualize the monomer units we will use to make our protein

In [ ]:
from pBuilder import Arginine, Glycine, Glutamine, Leucine, Alanine # R, G, Q, L, A

mol = Glycine() # Feel free to replace this with the different amino acids listed above to see what they look like
mol.visualize()

Here we will build up a protein based on the MaSp1 spider silk protein

In [ ]:
n_repeats = 2
chain = Protein()
nonrepeat = 'GGQGGAGQGGYGGLGSQGAGRGGLGGQ'
repeat = 'GAGAAAAAAGGAGQGGTGGLGSQGAGRGGL'
chain.name="Protein"
chain.build(nonrepeat+repeat*n_repeats)
chain.translate(-chain.center) #translate_to, rotate, spin, rotate_dihedral
chain.visualize()


In [ ]:
rotable_bond = list(chain.bonds())[301]
chain.rotate_dihedral(bond=rotable_bond, phi=3.14) #translate_to, rotate, spin, rotate_dihedral
# control energy minimization
chain.visualize()

In [ ]:
water = mb.load("O", smiles=True)
water.name="H2O"
water_box = mb.fill_box(water, box=[5,5,5], n_compounds=100)
water.visualize()

## 3. Combine and Solvate
---

In [ ]:
packed_box = mb.fill_box([chain, water], n_compounds=[1,1000], box=[10,10,10])
packed_box.print_hierarchy()
packed_box.visualize() #doesn't show anything, is this a bug?

In [ ]:
# save out and reload current state for future use
packed_box.save("solvated_protein.pdb", overwrite=True)
reloaded_pdb = mb.load("solvated_protein.pdb") #xyz, gro, lammpsdata, sdf, mol2, hoomdxml, json

In [ ]:
! head "solvated_protein.pdb"

## 4. Load Two ForceFields
---

A mosdef compatible implementation of the Generalized Amber Forcefield (GAFF) can be found [here](https://github.com/rsdefever/GAFF-foyer)

In [ ]:
gaff_forcefield = gmso.ForceField("../forcefields/gaff.xml")
gaff_forcefield

In [ ]:
tip3p_forcefield = gmso.ForceField("../forcefields/tip3p.xml")
tip3p_forcefield.combining_rule = "geometric"
tip3p_forcefield

## 5. Apply Forcefield
---

In [ ]:
from gmso.parameterization import apply

gmso_top = packed_box.to_gmso()
forcefield_matchingDict = {"Protein":gaff_forcefield, "H2O":tip3p_forcefield}
gmso_top = packed_box.to_gmso()
parameterized_top = apply(
    gmso_top, forcefield_matchingDict, identify_connections=True, 
) #Angles, dihedrals missing

### What Happened
This error indicates that have particles in our mbuild system that are missing parameters in our xml forcefield file. We will show how to correct this below.

## 6. Identify missing parameters
---

In [ ]:
gmso_top = packed_box.to_gmso()
parameterized_top = apply(
    gmso_top, forcefield_matchingDict, identify_connections=True, 
    assert_angle_params=False, assert_dihedral_params=False
)

In [ ]:
missing_angles = set()
for angle in gmso_top.angles:
    if angle.angle_type is None:
        missing_angles.add(angle.connection_members)
        for i in range(3):
            site = angle.connection_members[i]
            print(site.atom_type.name, site.atom_type.description, site.residue)
        print("###\n")

In [ ]:
# make a copy of gaff.xml and add the following line to the section labeled HarmonicAngleForce:
"""
<Angle class1="c1" class2="c3" class3="n2" angle="1.8242181341844732" k="836.8000000000001"/>
<Angle class1="c13" class2="c1" class3="oh" angle="1.6414931920303" k="300.8000000000001"/>
"""
# TODO: get changed doi for this forcefield, or version, and make a note
# TODO: use a gmso forcefield xml, not a foyer xml
# TODO: shrink forcefield to just necessary components

## 7. Reload and apply the forcefield
---

In [ ]:
gaff_forcefield = gmso.ForceField("../forcefields/gaff2.xml")
gmso_top = packed_box.to_gmso()
forcefield_matchingDict = {"Protein":gaff_forcefield, "H2O":tip3p_forcefield}
parameterized_top = apply(
    gmso_top, forcefield_matchingDict, identify_connections=True, 
)

## 7. Write HOOMD Objects
---

In [ ]:
#parameterized_top.save("top.gsd", overwrite=True)
import unyt as u

from gmso.external import to_hoomd_forcefield, to_hoomd_snapshot

base_units = {
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}

gmso_snapshot, snapshot_base_units = to_hoomd_snapshot(
    parameterized_top, base_units=base_units
)
gmso_forces, forces_base_units = to_hoomd_forcefield( #can't handle dimensionless parameters currently, PR incoming
    parameterized_top,
    r_cut=1.4,
    base_units=base_units,
    pppm_kwargs={"resolution": (64, 64, 64), "order": 7},
)
gmso_forces

## 8. Run HOOMD Simulations
---

In [ ]:
# Uses HOOMD 3
# This won't work until 7 works, but this does work on an ethanol box
import hoomd
temp = 300 * u.K
kT = temp.to_equivalent("kJ/mol", "thermal").value

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
# sim.create_state_from_gsd("top.gsd") # does not work
sim.create_state_from_snapshot(gmso_snapshot)
sim.operations.integrator = hoomd.md.Integrator(dt=0.001)
sim.operations.integrator.forces.extend(
    list(set().union(*gmso_forces.values()))[:-1]
)

nvt = hoomd.md.methods.NVT(
    kT=kT, tau=1.0, filter=hoomd.filter.All()
)
sim.operations.integrator.methods.append(nvt)

sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All()
)

sim.operations.computes.append(thermodynamic_properties)
logger = hoomd.logging.Logger()
logger.add(thermodynamic_properties)
import os
if os.path.exists('trajectory.gsd'):
    os.remove("trajectory.gsd")
gsd_writer = hoomd.write.GSD(
    filename='trajectory.gsd',
    trigger=hoomd.trigger.Periodic(1000),
     mode='xb',
     filter=hoomd.filter.All(),
     logger=logger
)
sim.operations.writers.append(gsd_writer)
outlogger = hoomd.logging.Logger(categories=['scalar', 'string'])
outlogger.add(sim, quantities=['timestep', 'tps'])
outlogger.add(thermodynamic_properties, ['kinetic_temperature'])
table = hoomd.write.Table(
    trigger=hoomd.trigger.Periodic(period=100),
    logger=outlogger
)
sim.operations.writers.append(table)
sim.run(100)

## 9. Analyze results
---

In [ ]:
import gsd
data = gsd.hoomd.read_log('trajectory.gsd')
timestep = data['configuration/step']
potential_energy = data[
    'log/md/compute/ThermodynamicQuantities/potential_energy']

fig, ax = plt.subplots(1,1,figsize=(5, 3.09))
ax.plot(timestep, potential_energy)
ax.set_xlabel('timestep')
ax.set_ylabel('potential energy')
fig.show()

